在這個工作坊內，你將會了解到 function calling 的基礎架構，並且你需要完成 todo 來實現缺少的工具，需要特別注意的是， api 一天只能呼叫 25 次

請先前往 https://www.alphavantage.co/support/#api-key 申請 api key

參考自官方範例，版權歸屬 Google

https://github.com/GoogleCloudPlatform/generative-ai/blob/25fc2ee894cab43733573f8f463cd3235bd72b64/gemini/function-calling/use_case_company_news_and_insights.ipynb

In [ ]:
!pip install --upgrade --quiet google-cloud-aiplatform requests


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import requests
from IPython.display import display, Markdown
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    Part,
    Tool,
)

In [ ]:
get_stock_price = FunctionDeclaration(
    name="get_stock_price",
    description="Fetch the current stock price of a given company",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company",
            }
        },
    },
)

# todo Step 1, implement company overview function declaration
# 可以參考前後的語法和文件來完成
# doc: https://www.alphavantage.co/documentation/#company-overview

get_company_news = FunctionDeclaration(
    name="get_company_news",
    description="Get the latest news headlines for a given company.",
    parameters={
        "type": "object",
        "properties": {
            "tickers": {
                "type": "string",
                "description": "Stock ticker symbol for a company",
            }
        },
    },
)

get_news_with_sentiment = FunctionDeclaration(
    name="get_news_with_sentiment",
    description="Gets live and historical market news and sentiment data",
    parameters={
        "type": "object",
        "properties": {
            "news_topic": {
                "type": "string",
                "description": """News topic to learn about. Supported topics
                               include blockchain, earnings, ipo,
                               mergers_and_acquisitions, financial_markets,
                               economy_fiscal, economy_monetary, economy_macro,
                               energy_transportation, finance, life_sciences,
                               manufacturing, real_estate, retail_wholesale,
                               and technology""",
            },
        },
    },
)

In [ ]:
# todo Step 2, 請將 step 1 實現的功能加入下方的 function_declarations 陣列之中

company_insights_tool = Tool(
    function_declarations=[
        get_stock_price,
        get_company_news,
        get_news_with_sentiment,
    ],
)

In [ ]:
API_KEY = "PASTE_YOUR_API_KEY_HERE" # @param {type:"string"}


In [ ]:
def get_stock_price_from_api(content):
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={content['ticker']}&apikey={API_KEY}"
    api_request = requests.get(url)
    return api_request.text

# todo Step 3, implement company overview function api request
# doc: https://www.alphavantage.co/documentation/#company-overview


def get_company_news_from_api(content):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={content['tickers']}&limit=20&sort=RELEVANCE&apikey={API_KEY}"
    api_response = requests.get(url)
    return api_response.text


def get_news_with_sentiment_from_api(content):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics={content['news_topic']}&limit=20&sort=RELEVANCE&apikey={API_KEY}"
    api_request = requests.get(url)
    return api_request.text

In [ ]:
# todo Step 4, 請將 step 3 實現的功能和 step 1 定義好的 name 加入下方的 handler 之中
function_handler = {
    "get_stock_price": get_stock_price_from_api,
    "get_company_news": get_company_news_from_api,
    "get_news_with_sentiment": get_news_with_sentiment_from_api,
}

In [ ]:
gemini_model = GenerativeModel(
    "gemini-1.5-flash-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[company_insights_tool],
)

In [ ]:
chat = gemini_model.start_chat()


In [ ]:

def send_chat_message(prompt):
    display(Markdown("#### Prompt"))
    print(prompt, "\n")
    prompt += """
    Give a concise, high-level summary. Only use information that you learn from
    the API responses.
    """

    # Send a chat message to the Gemini API
    response = chat.send_message(prompt)

    # Handle cases with multiple chained function calls
    function_calling_in_process = True
    while function_calling_in_process:
        # Extract the function call response
        function_call = response.candidates[0].content.parts[0].function_call

        # Check for a function call or a natural language response
        if function_call.name in function_handler.keys():
            # Extract the function call
            function_call = response.candidates[0].content.parts[0].function_call

            # Extract the function call name
            function_name = function_call.name
            display(Markdown("#### Predicted function name"))
            print(function_name, "\n")

            # Extract the function call parameters
            params = {key: value for key, value in function_call.args.items()}
            display(Markdown("#### Predicted function parameters"))
            print(params, "\n")

            # Invoke a function that calls an external API
            function_api_response = function_handler[function_name](params)[
                :20000
            ]  # Stay within the input token limit
            display(Markdown("#### API response"))
            print(function_api_response[:500], "...", "\n")

            # Send the API response back to Gemini, which will generate a natural language summary or another function call
            response = chat.send_message(
                Part.from_function_response(
                    name=function_name,
                    response={"content": function_api_response},
                ),
            )
        else:
            function_calling_in_process = False

    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\$")))

延伸閱讀
ReAct
https://github.com/GoogleCloudPlatform/generative-ai/blob/25fc2ee894cab43733573f8f463cd3235bd72b64/gemini/function-calling/intro_diy_react_agent.ipynb

測試功能

In [ ]:
stock_price_prompt = "What is the current stock price for Google?"
overview_prompt = "What is the company overview for Google?"
multiple_overview_prompt = "Give me a company overview of Walmart and The Home Depot"
news_prompt = "What are the latest news headlines for Google?"
industry_news_prompt = "Has there been any exciting news related to real estate recently?"

In [ ]:
send_chat_message()